In [18]:
import os
import sys

spark_path = "C:\spark"

os.environ['SPARK_HOME'] = spark_path
os.environ['HADOOP_HOME'] = spark_path

sys.path.append(spark_path + "/bin")
sys.path.append(spark_path + "/python")
sys.path.append(spark_path + "/python/pyspark/")
sys.path.append(spark_path + "/python/lib")
sys.path.append(spark_path + "/python/lib/pyspark.zip")
sys.path.append(spark_path + "/python/lib/py4j-0.10.4-src.zip")

from pyspark import SparkContext
from pyspark import SparkConf
from pyspark import SQLContext

In [19]:
sc=SparkContext(appName="pracaDypl", master="local[*]")

ValueError: Cannot run multiple SparkContexts at once; existing SparkContext(app=pracaDypl, master=local[*]) created by __init__ at <ipython-input-2-95e70462ced8>:1 

In [10]:
rdd = sc.textFile("C:\\spark\README.md")

In [11]:
rdd.count()

103

In [13]:
sc.startTime

1524219795100

## 1. Import danych. 

In [ ]:
# Dane, na których będziemy pracować to dump forum Stack Exchange. 
# Dane są dostępne pod adresem "https://archive.org/details/stackexchange".
# Poszczególne działy forum są udostępnione w postaci plików XML. 

# Do dalszej analizy wybrano dział: Datascience, plik: Posts. W pliku znjdują się szczegółowe informacje odnośnie wpisów
# na forum, poczynając od id, daty, powiązań z innymi wpisami, na treści wpisu kończąc. Celem jest wyciągnięcie samej treści 
# pytań.

## Metoda pierwsza

In [20]:
# bezpośrednio wczytujemy plik XML do Sparkowego RDD 
lines = sc.textFile("file:///Users/Spal/Desktop/Praca dyplomowa/Dane/Posts.xml")
# # Czy ścieżkę dostępu da się zwinąć do zmiennej? a może sam url? jest jakiś problem ze znakami z XMLa, zamienia na papkę 

In [9]:
lines.take(4)

['<?xml version="1.0" encoding="utf-8"?>',
 '<posts>',
 '  <row Id="5" PostTypeId="1" CreationDate="2014-05-13T23:58:30.457" Score="8" ViewCount="412" Body="&lt;p&gt;I\'ve always been interested in machine learning, but I can\'t figure out one thing about starting out with a simple &quot;Hello World&quot; example - how can I avoid hard-coding behavior?&lt;/p&gt;&#xA;&#xA;&lt;p&gt;For example, if I wanted to &quot;teach&quot; a bot how to avoid randomly placed obstacles, I couldn\'t just use relative motion, because the obstacles move around, but I don\'t want to hard code, say, distance, because that ruins the whole point of machine learning.&lt;/p&gt;&#xA;&#xA;&lt;p&gt;Obviously, randomly generating code would be impractical, so how could I do this?&lt;/p&gt;&#xA;" OwnerUserId="5" LastActivityDate="2014-05-14T00:36:31.077" Title="How can I do simple machine learning without hard-coding behavior?" Tags="&lt;machine-learning&gt;" AnswerCount="1" CommentCount="1" FavoriteCount="1" Closed

In [12]:
lines.count()

19216

In [ ]:
# Poza pierwszymi dwoma wierszami, jeden wiersz to komplet informacji na temat jednego pytania lub odpowiedzi (dlaczego 
# mamy tylko nieparzyste row Id?)

In [ ]:
# Jak usunąć dwa pierwsze wiersze z RDD? filter, ale może da się łatwiej? 

In [21]:
posts = lines.filter(lambda x: "row Id" in x)

In [11]:
posts.take(3)

['  <row Id="5" PostTypeId="1" CreationDate="2014-05-13T23:58:30.457" Score="8" ViewCount="412" Body="&lt;p&gt;I\'ve always been interested in machine learning, but I can\'t figure out one thing about starting out with a simple &quot;Hello World&quot; example - how can I avoid hard-coding behavior?&lt;/p&gt;&#xA;&#xA;&lt;p&gt;For example, if I wanted to &quot;teach&quot; a bot how to avoid randomly placed obstacles, I couldn\'t just use relative motion, because the obstacles move around, but I don\'t want to hard code, say, distance, because that ruins the whole point of machine learning.&lt;/p&gt;&#xA;&#xA;&lt;p&gt;Obviously, randomly generating code would be impractical, so how could I do this?&lt;/p&gt;&#xA;" OwnerUserId="5" LastActivityDate="2014-05-14T00:36:31.077" Title="How can I do simple machine learning without hard-coding behavior?" Tags="&lt;machine-learning&gt;" AnswerCount="1" CommentCount="1" FavoriteCount="1" ClosedDate="2014-05-14T14:40:25.950" />',
 '  <row Id="7" Pos

In [13]:
# zniknęły trzy wiersze, a nie tylko pierwsze dwa... jeszcze pewnie na końcu coś wypadło
questions.count()


19213

In [ ]:
# Czyli mamy w dziale DataScience 19213 pytań i dopowiedzi.

In [ ]:
# Niestety nie potrafię z takiegoe RDD wyciąć kawałka, który sobie założyłem na początku...

## Metoda Druga

In [ ]:
# Drugi pomysł, to użycie pakietu w Pysparku do importu xml.

In [3]:
from pyspark.sql import SQLContext
sqlContext = SQLContext(sc)

In [4]:
%AddDeps com.databricks spark-csv_2.10 1.2.0 --transitive

ERROR:root:Line magic function `%AddDeps` not found.


In [6]:
df = sqlContext.read.format('com.databricks.spark.xml').options(rowTag='user'). \
load("file:///Users/Spal/Desktop/Praca dyplomowa/XML testing/Users1.xml")

Py4JJavaError: An error occurred while calling o47.load.
: java.lang.ClassNotFoundException: Failed to find data source: com.databricks.spark.xml. Please find packages at http://spark.apache.org/third-party-projects.html
	at org.apache.spark.sql.execution.datasources.DataSource$.lookupDataSource(DataSource.scala:546)
	at org.apache.spark.sql.execution.datasources.DataSource.providingClass$lzycompute(DataSource.scala:87)
	at org.apache.spark.sql.execution.datasources.DataSource.providingClass(DataSource.scala:87)
	at org.apache.spark.sql.execution.datasources.DataSource.resolveRelation(DataSource.scala:302)
	at org.apache.spark.sql.DataFrameReader.load(DataFrameReader.scala:178)
	at org.apache.spark.sql.DataFrameReader.load(DataFrameReader.scala:156)
	at sun.reflect.NativeMethodAccessorImpl.invoke0(Native Method)
	at sun.reflect.NativeMethodAccessorImpl.invoke(NativeMethodAccessorImpl.java:62)
	at sun.reflect.DelegatingMethodAccessorImpl.invoke(DelegatingMethodAccessorImpl.java

In [37]:
sc.version # która wersja sparka, Scala jest w wersji 2.11.8

'2.2.1'

In [ ]:
# Sypie błędami i nie umiem tego przeskoczyć.

## Metoda trzecia.

In [ ]:
# Kolejny pomysł, to użycie pakietu spark xml bezpośrednio w scali. W notatniku wpisuję treść poleceń, które należy przekleić
# do cmd.

In [ ]:
# Należy uruchomić spark shell w trybie xml. Cyferki na końcu to wersja scali, która ma odpowiadać zainstalowanej.

In [ ]:
spark-shell --packages com.databricks:spark-xml_2.11:0.4.1

In [ ]:
# Następnie

In [ ]:
# Zaimportować sql context
import org.apache.spark.sql.SQLContext
# Dodac sql context
val sqlContext = new org.apache.spark.sql.SQLContext(sc)
# Wczytanie pliku, utworzenie data framu, jest problem z prawidłowym podaniem ścieżki pliku
val df = sqlContext.read.format("com.databricks.spark.xml").option("rowTag", "user").load("file:///Users/Spal/Desktop/Praca dyplomowa/Dane/Users.xml")
val df4 = sqlContext.read.format("com.databricks.spark.xml").schema(customSchema).load("C://Users/Spal/Desktop/Praca dyplomowa/Dane/Posts.xml")
val df5 = sqlContext.read.format("com.databricks.spark.xml").load("C://Users/Spal/Desktop/Praca dyplomowa/XML testing/Users.xml")
#podgląd 
df.show()

In [ ]:
# wychodzi pusta data frame bez informacji o błedzie... nie wiem co jest nie tak... format pliku xml trochę inny niż wymagany?
# brak zamknięcia </row> ? 
# Problem wynika z tego, że w dokumenatcji mamy "At the moment, rows containing self closing xml tags are not supported"...
# czyli XML z danymi jest akurat w takiej postaci, której nie rozumie 

In [ ]:
# Znalazłem obejście - przekształcenie pliku XML do postaci zjadliwej dla spark xml
# trzeba utworzyć plik xsl, w którym definiujemy nową strukturę xml 
# w pliku xml trzeba zapisać odniesienie do xls
# po otwarciu za pomocą word zapisuje jako oddzielny plik xml
# trzeba go pozniej otworzyc w notatniku i wyciać odniesienei do xsla 
# i to już jest plik, który spark łyknie

In [16]:
## Metoda czwarta

In [ ]:
# bezpośrednio w Scali 

In [ ]:
# piszemy
import xml._
XML.loadFile("C://Users/Spal/Desktop/Praca dyplomowa/Dane/Posts.xml")
# i cały plik się wyświetli w konsoli, co mi z tego? nie mam pojęcia
# wyciąganie zawartości
res1 \ "row"
res2 \\"@Body" # to działa, ale czy to jest BIG DATA czy tylko programowanie w Scali?

# 2. Analiza zbioru. Proste transformacje.

## Metoda pierwsza cd.

In [ ]:
# Potrzebna metoda na wyfiltrowanie konkretnego fragmentu informacji z wiersza.

In [22]:
# Jak dobrać się do konkretnej porcji danych? Pierwszy pomysł: map, split i rozdzielenie spacja? źle...
fields = posts.map(lambda x: x.split(' '))

In [23]:
fields.take(3)

[['',
  '',
  '<row',
  'Id="5"',
  'PostTypeId="1"',
  'CreationDate="2014-05-13T23:58:30.457"',
  'Score="8"',
  'ViewCount="412"',
  'Body="&lt;p&gt;I\'ve',
  'always',
  'been',
  'interested',
  'in',
  'machine',
  'learning,',
  'but',
  'I',
  "can't",
  'figure',
  'out',
  'one',
  'thing',
  'about',
  'starting',
  'out',
  'with',
  'a',
  'simple',
  '&quot;Hello',
  'World&quot;',
  'example',
  '-',
  'how',
  'can',
  'I',
  'avoid',
  'hard-coding',
  'behavior?&lt;/p&gt;&#xA;&#xA;&lt;p&gt;For',
  'example,',
  'if',
  'I',
  'wanted',
  'to',
  '&quot;teach&quot;',
  'a',
  'bot',
  'how',
  'to',
  'avoid',
  'randomly',
  'placed',
  'obstacles,',
  'I',
  "couldn't",
  'just',
  'use',
  'relative',
  'motion,',
  'because',
  'the',
  'obstacles',
  'move',
  'around,',
  'but',
  'I',
  "don't",
  'want',
  'to',
  'hard',
  'code,',
  'say,',
  'distance,',
  'because',
  'that',
  'ruins',
  'the',
  'whole',
  'point',
  'of',
  'machine',
  'learning.&lt;/p&

In [ ]:
# Porcja danych ma zawsze postać XX="....". Dla danego XX chciałbym wydobyć fragment znajdujący się pomiędzy " ".

In [ ]:
# czyli jakoś tak, pożniej odliczyć na którym miejscu jest interesujące nas pole i ognia

In [24]:
fields = posts.map(lambda x: x.split('"'))

In [27]:
fields.take(1)

[['  <row Id=',
  '5',
  ' PostTypeId=',
  '1',
  ' CreationDate=',
  '2014-05-13T23:58:30.457',
  ' Score=',
  '8',
  ' ViewCount=',
  '412',
  ' Body=',
  "&lt;p&gt;I've always been interested in machine learning, but I can't figure out one thing about starting out with a simple &quot;Hello World&quot; example - how can I avoid hard-coding behavior?&lt;/p&gt;&#xA;&#xA;&lt;p&gt;For example, if I wanted to &quot;teach&quot; a bot how to avoid randomly placed obstacles, I couldn't just use relative motion, because the obstacles move around, but I don't want to hard code, say, distance, because that ruins the whole point of machine learning.&lt;/p&gt;&#xA;&#xA;&lt;p&gt;Obviously, randomly generating code would be impractical, so how could I do this?&lt;/p&gt;&#xA;",
  ' OwnerUserId=',
  '5',
  ' LastActivityDate=',
  '2014-05-14T00:36:31.077',
  ' Title=',
  'How can I do simple machine learning without hard-coding behavior?',
  ' Tags=',
  '&lt;machine-learning&gt;',
  ' AnswerCount=',
